In [1]:
from sklearn import svm, preprocessing, metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

In [2]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import random

In [3]:
#a = all_data[0]
# a.head()

In [4]:
# a.columns

In [5]:
def AU(n):
    # return AU column name
    return f'AU{str(n).zfill(2)}_r'

In [6]:
# define useful functions

count = 0
RANGE = 1000
def closest_frame(df, f):
    f_near = f
    for x in range(50):
        while len(face_data.loc[face_data['frame'] == f_near, 'milis'].values) == 0:
            f_near -= 1
#     print("number of frames retraced: " + str(f - f_near))
    return f_near

def df_near(df, frame, ms_range=RANGE):
    """return rows where milis is within the search range"""
    if frame is None:
        return None
    ms = df.loc[df['frame'] == closest_frame(df,frame), 'milis'].values[0]
    locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms + ms_range)]
#     locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms)]
    return locs

def max_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.max()).transpose()

def min_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.min()).transpose()

def mean_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.mean()).transpose()

def sd_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.std()).transpose()

In [7]:
# gather data

# choose which action units to look at
AUs_to_look_at = [9, 14, 15, 17, 20, 23]

def get_pid(filename):
    """ get pid from dirname """
    return filename.split('_')[0][1:]
def get_mode(filename):
    """ get mode from dirname """
    return int(file.split('_')[2][1])

up_data_max = []
down_data_max = []
up_data_min = []
down_data_min = []
all_data = []

up_data_sd = []
down_data_sd = []

up_data_mean = []
down_data_mean = []

all_data_max = []
all_data_min = []
all_data_mean = []
all_data_sd = []

PATH = "/Users/yoony/Desktop/si_research/face_analysis/implicit_feedback_si_faceanalysis/processed"
csvs = []
for file in sorted(os.listdir(PATH)):
    if file[0] == '.':
        continue
    print(file)
#     csvs.append(pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv')))
    data = pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv'))
    face_data = data.drop(columns=list(data.columns.values[2:678]))
    face_data = face_data.drop(columns=list(face_data.columns.values[19:]))
    face_data = face_data[(['frame', 'milis'] + [AU(x) for x in AUs_to_look_at])]
    
    signals = pd.read_csv('../my_buckets.csv')[['player_id', 'mode', 'signal_type', 'frame_number', 'code']]
    s1 = signals.loc[(signals['player_id'] == get_pid(file)) & (signals['mode'] == get_mode(file))]
    s_up = s1.loc[s1['signal_type'] == 'up'].frame_number.values
    s_down = s1.loc[s1['signal_type'] == 'down'].frame_number.values
    
    for x in s_up:
        up_locs = df_near(face_data, x)
        up_data_max.append(max_df(up_locs))
        up_data_min.append(min_df(up_locs))
        up_data_sd.append(sd_df(up_locs))
        up_data_mean.append(mean_df(up_locs))
        
        all_data_max.append(max_df(up_locs))
        all_data_min.append(min_df(up_locs))
        all_data_sd.append(sd_df(up_locs))
        all_data_mean.append(mean_df(up_locs))
        
    for x in s_down:
        down_locs = df_near(face_data, x)
        down_data_max.append(max_df(down_locs))
        down_data_min.append(min_df(down_locs))
        down_data_sd.append(sd_df(down_locs))
        down_data_mean.append(mean_df(down_locs))
        
        all_data_max.append(max_df(down_locs))
        all_data_min.append(min_df(down_locs))
        all_data_sd.append(sd_df(down_locs))
        all_data_mean.append(mean_df(down_locs))
        
    all_data.append(face_data)
    
    

P5ee6d2494fd04045b988a12d_voff_m1_g1_t2021_7_10_0_47
P5ee6d2494fd04045b988a12d_voff_m2_g2_t2021_7_10_0_50
P5ef33c05f4c5240413ef9ac2_voff_m1_g1_t2021_7_13_20_38
P5ef33c05f4c5240413ef9ac2_voff_m2_g2_t2021_7_13_20_34
P6081c511ded5945d9b70801e_voff_m1_g1_t2021_7_13_21_6
P6081c511ded5945d9b70801e_voff_m2_g2_t2021_7_13_21_11
P60d520539b147c4acaf44b77_voff_m1_g1_t2021_7_10_1_4
P60d520539b147c4acaf44b77_voff_m2_g2_t2021_7_10_1_1
P60e5e62659dc1912bc43ed35_voff_m1_g1_t2021_7_9_23_29
P60e5e62659dc1912bc43ed35_voff_m2_g2_t2021_7_9_23_24
P60e9e9618953d7a3430dd8f6_voff_m1_g1_t2021_7_13_18_51
P60e9e9618953d7a3430dd8f6_voff_m2_g2_t2021_7_13_18_57
P60edb7015a9231789057bdea_voff_m1_g1_t2021_7_13_21_7
P60edb7015a9231789057bdea_voff_m2_g2_t2021_7_13_21_11


In [8]:
def rename_columns(df, tag):
    a = df.copy()
    a.columns = list(map(lambda x: x + tag, a.columns))
    return a

def merge_dfs(dfs):
    a = dfs[0]
    for d in dfs[1:]:
        a = a.join(d, how='outer')
    return a
        

In [9]:
u1 = rename_columns(pd.concat(up_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
u2 = rename_columns(pd.concat(up_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
u3 = rename_columns(pd.concat(up_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
u4 = rename_columns(pd.concat(up_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [10]:
# ups = merge_dfs([u1, u2, u3, u4])
ups = merge_dfs([u1,u2,u3,u4])
ups['type'] = 'up'

In [11]:
d1 = rename_columns(pd.concat(down_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
d2 = rename_columns(pd.concat(down_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
d3 = rename_columns(pd.concat(down_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
d4 = rename_columns(pd.concat(down_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [12]:
# downs = merge_dfs([d1, d2, d3, d4])
downs = merge_dfs([d1,d2,d3,d4])
downs['type'] = 'down'

In [13]:
means = pd.DataFrame(pd.concat(up_data_mean).mean(), columns=['up_mean'])
means['down_mean'] = pd.concat(down_data_mean).mean()
means['mean_diff'] = means['up_mean'] - means['down_mean']
means['up_sd'] = pd.concat(up_data_max).std()
means['down_sd'] = pd.concat(down_data_max).std()
means['sd_diff'] = means['up_sd'] - means['down_sd']
means = means.drop(index=['frame', 'milis'])
# means['total'] = pd.concat(all_data).mean
# means.to_csv('face/au_data.csv')
means

,up_mean,down_mean,mean_diff,up_sd,down_sd,sd_diff
AU01_r,0.858121,0.777835,0.080285,0.710110,0.561180,0.148930
AU02_r,0.730789,0.705810,0.024979,0.888196,0.832287,0.055908
AU04_r,0.516726,0.538601,-0.021875,0.516376,0.734489,-0.218113
AU05_r,0.544575,0.537046,0.007529,0.691678,0.502041,0.189637
AU06_r,0.088434,0.009912,0.078522,0.380299,0.129482,0.250817
AU07_r,0.113406,0.056134,0.057272,0.445741,0.296787,0.148954
AU09_r,0.393464,0.099633,0.293831,0.798078,0.252278,0.545800
AU10_r,0.060520,0.028240,0.032280,0.423664,0.176865,0.246798
AU12_r,0.030093,0.018152,0.011941,0.224556,0.119350,0.105206
AU14_r,0.375166,0.177727,0.197439,0.690711,0.541141,0.149570


In [14]:
# check number of signals
len(up_data_max) + len(down_data_max)

86

In [76]:
clf = svm.SVC(kernel='linear')

In [77]:
#clean up and finalize the data
signal_data = pd.concat([ups, downs]).reset_index(drop=True)
signal_data
signal_types = signal_data['type'].values
signal_data = signal_data.drop(columns=['type'])

In [78]:
X = signal_data
y = signal_types

In [79]:
# # split up into testing and training
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109)

In [80]:
# normalize first
def result(over_type):
    seed = 109 #random seed
    seed = random.randint(0,200)
    
    over = { 'random': RandomOverSampler(random_state=seed),
             'smote': SMOTE(random_state=seed),
             'adasyn': ADASYN(random_state=seed)}
    
#     ros = RandomOverSampler(random_state=seed)
#     smote = SMOTE(random_state=seed)
#     adasyn = ADASYN(random_state=seed)
    
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

    # normalize inputs, use fit_transform for train and transform for test
    normalizer = preprocessing.Normalizer()
    norm_X_train = normalizer.fit_transform(X_train)
    norm_X_test = normalizer.transform(X_test)

    X_train_resampled, y_train_resampled = over[over_type].fit_resample(norm_X_train, y_train)
    # X_train_resampled, y_train_resampled = under.fit_resample(X_train_resampled, y_train_resampled)

    # X_train_resampled, y_train_resampled = smote.fit_resample(norm_X_train, y_train)
    # X_train_resampled, y_train_resampled = X_train, y_train


#     counter = Counter(y_train)
#     print('original: ' + str(counter))
#     print('resampled: ' + str(Counter(y_train_resampled)))

    # fit data
    clf.fit(X_train_resampled, y_train_resampled)
    # predict
    y_pred = clf.predict(norm_X_test)
    return metrics.f1_score(y_test, y_pred, average=None)

In [81]:
# for i in range(100):
#     result()[0]
r = [result('random')[0] for i in range(100)]
s = [result('smote')[0] for i in range(100)]
a = [result('adasyn')[0] for i in range(100)]
print(f'random:  {np.mean(r)}')
print(f'smote:  {np.mean(s)}')

print(f'adasyn:  {np.mean(a)}')


random:  0.36561746745474705
smote:  0.3762486581455218
adasyn:  0.368724706040583


In [82]:
# r = pd.DataFrame(columns=['test', 'pred'])
# r['test'] = y_test
# r['pred'] = y_pred
# r